# はじめに
* DeepNeuralNetを、TensorFlow等のライブラリを用いずにスクラッチで実装していく。
* ここでは全結合5層の、画像の判別機をつくる。（CNNは用いない）
* ある程度の知識は前提として進める。DeepLearningや微積分の基本等。
* 量が多くなるため、随時更新。

In [4]:
import keras

Using TensorFlow backend.
/Users/takizawa/dev/python/play/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
from keras.datasets import cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)

# 1. 重みの初期化
* 重みを初期化するメソッドを作成する
* DeepLearningで学習を進める前に、W,bを初期化する必要がある。
* Wは0で初期化すると、学習が進まない。
* 最適な初期値については、Xavier,Heの初期値、といったものがあるが、ここでは単純に微小なランダムの値とする。

In [1]:
import numpy as np
np.random.seed(1)

In [2]:
def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1,L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dims[l],1))
        
    return parameters

* 上記は`layer_dims`に作成するDLの各層のユニット数を指定することで、Wとbを返してくれる。
* 例えば、inputは5、outputは1、隠れ層1層でユニット数3とすると下記。

In [3]:
initialize_parameters([5,3,1])

{'W1': array([[ 0.01624345, -0.00611756, -0.00528172, -0.01072969,  0.00865408],
        [-0.02301539,  0.01744812, -0.00761207,  0.00319039, -0.0024937 ],
        [ 0.01462108, -0.02060141, -0.00322417, -0.00384054,  0.01133769]]),
 'W2': array([[-0.01099891, -0.00172428, -0.00877858]]),
 'b1': array([[ 0.],
        [ 0.],
        [ 0.]]),
 'b2': array([[ 0.]])}

* W1,W2,b1,b2を作成してくれる。

# 2. Forward Propagation
* Forward Propagationを実装する
* まずは1つ前のActivation Functionの返り値とW,bを基に、活性化関数に入れる数値を計算。

In [2]:
def linear_forward(A,W,b):
    # 1つ前のAにWをかけて、bを足し、Zを求める
    Z = np.dot(W,A) + b
    
    # Backwardを計算する際に用いる
    cache = (A,W,b)
    
    return Z, cache

* 活性化関数を作成する。
* ReLU関数とsigmoid関数をつくる。

In [5]:
def relu(Z):
    A = Z * (Z > 0)
    return A

In [6]:
def sigmoid(Z):
    A = 1 / (1 + np.exp(Z))
    return A, Z

In [15]:
sigmoid(np.array([1,1,1,2,2,])).reshape(-1,1)

array([[ 0.26894142],
       [ 0.26894142],
       [ 0.26894142],
       [ 0.11920292],
       [ 0.11920292]])

* ここまでに作成した、ReLU関数、sigmoid関数、linear_forward関数を元に、前層のActicationの返り値から次のActivationの返り値を返す関数を作る

In [3]:
def linear_activation_forward(A_prev, W, b,activation):

    Z, linear_cache = linear_forward(A_prev,W,b)

    if activation == "relu":
        A, activation_cache = relu(Z)
    
    elif activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
    
    cache = (linear_cache, activation_cache)
    
    return A, cache

* ここまでで各層での計算の関数はできたため、最後にinputからoutputを返す関数を作成する

In [8]:
def model_forward(X, parameters):
    
    caches = []
    A = X
    # 各層の間にW,bがあるため、数を2で割れば層数になる
    L = len(parameters) // 2 
    
    # 隠れ層のユニットではReLUを用いる
    for l in range(L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev,
                                             parameters["W"+str(l)],
                                             parameters["b"+str(l)],
                                             "relu")
        caches.append(cache)
    
    # 最終層はSigmoidを用いる
    AL, cache = linear_activation_forward(A,
                                          parameters["W"+str(L)],
                                          parameters["b"+str(L)],
                                          "sigmoid")
    caches.append(cache)
    
    return AL, caches

* 上記で、ForwardPropagationは実装できた

# 3. Cost Function
* ForwardPropagationによって、あるinputに対し、あるパラメータW、bの時の、予測値を求めることができるようになった。
* ここではその予測結果がどれだけ正しいのかを算出するために、Cost関数を作成する。
* クロスエントロピー誤差を用いる。

In [9]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    
    cost = - 1/m * (np.dot(Y, np.log(AL.T)) + np.dot(1-Y, np.log((1-AL).T)))
    
    # costのshapeを変更して、数値型で返すようにする。
    cost = np.squeeze(cost)
    
    return cost

* np.squeezeは下記のようなことをしてくれている

In [10]:
np.squeeze([[[[[13]]]]])

array(13)

# 4. Backward Propagation
* Backward Propagationを実装する。
* Z = WA+b のBackwardと、A = activation(Z)のBackwardを実装する必要がある。
* Forward Propagationでは、ReLUとSigmoidの活性化関数を実装したので、Backwardでも両方に対応する必要がある。

In [11]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

* ReLUとsigmoidの微分を作成する
* dZ = dA * g'(Z)

In [16]:
def relu_backward(dA, activation_cache):
    Z  = activation_cache
    dZ = dA * (Z > 0)
    
    return dZ

In [17]:
def sigmoid_backward(dA, activation_cache):
    Z = activation_cache
    dZ = dA * sigmoid(1 - Z)[0] * sigmoid(Z)[0]
    
    return dZ

* 上記の関数を用いて、1層分のBackward Propagationを作成する

In [1]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

* さらにこれを多層にする

In [1]:
def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = - ( np.divide(Y, AL) - np.divide(1-Y, 1-AL))
    
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+2)],
                                                                   current_cache,
                                                                   "relu")
        grads["dA" + str(l+1)] = dA_prev_temp
        grads["dW" + str(l+1)] = dW_temp
        grads["db" + str(l+1)] = db_temp
        
    return grads

* 上記で勾配を求められるようになったので、その勾配を元に、パラメータを更新していく

In [2]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2
    
    for l in range(1, L+1):
        parameters["W"+str(l)] -= learning_rate * grads["dW"+str(l)]
        parameters["b"+str(l)] -= learning_rate * grads["db"+str(l)]
        
    return parameters

* 続きます

## 参考
* [Coursera deeplearning.ai](https://www.coursera.org/learn/neural-networks-deep-learning/home/welcome)